In [1]:
# === SETUP === #

import pandas as pd
import numpy as np
import os
from google.colab import drive
import sys
from sklearn.preprocessing import StandardScaler

In [2]:
drive.mount('/content/drive')
folder = '/content/drive/MyDrive/Quantitative Investment Portfolio/'
data_dir = '/content/drive/MyDrive/Quantitative Investment Portfolio/Data/'
sys.path.append('/content/drive/MyDrive/Quantitative Investment Portfolio/Code/')
from ProspectTheory import ProspectTheory

Mounted at /content/drive


In [3]:
data = pd.read_parquet(data_dir + 'factors.parquet')

In [4]:
data.dropna(subset=['ret'],inplace=True)
data.sort_values(['permno','yyyymm'],inplace=True)
data.reset_index(drop=True,inplace=True)

In [9]:
1.01**(365)

37.78343433288728

In [10]:
np.exp(365*np.log(1.01))

37.78343433288729

In [16]:
A = data.groupby(['permno','yyyymm'])['ret'].count()

In [17]:
A[A>1]

,,ret
permno,yyyymm,


In [ ]:
# === Classify Industry Based on FamaFrench === #

indus = pd.read_excel(data_dir+'SIC_49_Industry.xlsx')
data['ind'] = 50

indus = indus.sort_values('SIC_start')
for _, row in indus.iterrows():
  mask = (data['hsiccd'] >= row['SIC_start']) & (data['hsiccd'] <= row['SIC_end'])
  data.loc[mask, 'ind'] = row['Industry']

In [ ]:
# === Add the Prospect Theory Factor === #

## Need to add the modified version of the factor to account for recency bias

In [ ]:
prospect = ProspectTheory()
prospect_df = prospect.prospect(data)
prospect_df = prospect_df[['permno','yyyymm','TK']]
data = pd.merge(data,prospect_df,on=['permno','yyyymm'],how='left')

Initialized ProspectTheory with:
α (alpha): 0.88
λ (lambda): 2.25
γ (gamma): 0.61
θ (theta): 0.69
Window: 36 months

--- Prospect Theory Factor Computation ---

--- Prospect Theory Computation Complete ---


In [ ]:
# === Add the Market Age Factor === #

data['market_age'] = (data['yyyymm'] - data.groupby('permno')['yyyymm'].transform('first'))//12

In [ ]:
# === Update the Factors file === #

data.to_parquet(data_dir + 'factors.parquet')

In [14]:
# === Divide the date in train and testing === #

data_divison = {'insample':[1993,2013],'outsample':[2014,2024],'presample':[1990,1992]}

In [15]:
data['s'] = 0
data['year'] = data['yyyymm']//100
data['month'] = data['yyyymm']%100
data.loc[data['year']>=data_divison['outsample'][0],'s'] = 1
data.loc[data['year']<=data_divison['presample'][1],'s'] = 2

In [18]:
def is_out(x):
    return abs(x - x.mean()) > x.std() * 3

In [23]:
def ubound(x,dx=0.01,dy=0.01):
    data_range = x.max() - x.min()
    near_max = x >= (x.max() - dx * data_range)  # 1% below max
    if(near_max.mean()>0.01):return 1
    return 0
def lbound(x,dx=0.01):
    data_range = x.max() - x.min()
    near_min = x <= (x.min() + dx * data_range)  # 1% above min
    if(near_min.mean()>0.01):return 1
    return 0

def handle_outliers(data,cols, q=[0.005,0.995], max_missing=2):
    data_clip = data.loc[:,cols].copy()
    data_clip = data_clip.loc[:,data.nunique()>10]
    l = data_clip.quantile(q[0],axis=0)
    u = data_clip.quantile(q[1],axis=0)

    cols = data_clip.apply(ubound)
    cols = cols[cols>0]
    u[cols.index] = np.inf

    cols = data_clip.apply(lbound)
    cols = cols[cols>0]
    l[cols.index] = -np.inf

    data_clip = data_clip.clip(lower=l, upper=data_clip.quantile(q[1],axis=0),axis='columns')
    outs = data_clip.apply(is_out)
    outB = outs.sum(axis=1) > max_missing
    data.loc[:,data_clip.columns] = data_clip
    data = data[~outB]

    return data

In [20]:
def fit_scaler(data, cols):
    cols_f = list(cols)
    cols_x = []
    train_group = data.copy()
    train_group[np.isinf(train_group)] = np.nan
    for i in cols:
        if train_group[i].count() < 2:
            cols_f.remove(i)
            cols_x.append(i)
        elif train_group[i].nunique() == 1:
            cols_f.remove(i)
            cols_x.append(i)
        elif np.nanstd(train_group[i], axis=0, ddof=0) in [np.nan, np.inf, -np.inf, 0]:
            cols_f.remove(i)
            cols_x.append(i)
    # Check if cols_f is empty before fitting the scaler
    if not cols_f:
        print("Warning: No valid columns for scaling in this group.")
        return None, []  # or raise an exception if desire
    scaler = StandardScaler().fit(train_group[cols_f])
    del train_group
    return scaler, cols_f, cols_x

def transform_group(group, scaler, cols_f):
    ret = group.copy()
    ret[cols_f] = scaler.transform(group[cols_f])
    return ret

In [24]:
# === Data Cleaning === #
df = data.copy()
cols = ['permno','yyyymm','ind','hsiccd','s','year','month']
cols = df.columns[~df.columns.isin(cols)]

print(f'Starting rows: {df.shape[0]}')
df = handle_outliers(df,cols)
print(f'Exclude Outliers: {df.shape[0]}')

# ? Drop Financials ? #
df = df[df['ind']!=48]
df = df[df['ind']!=50]
print(f'Drop Financials: {df.shape[0]}')

Starting rows: 2562651
Exclude Outliers: 2449595
Drop Financials: 1629397


In [26]:
B = df.groupby(['permno','yyyymm'])['ret'].count()
B[B>1]

,,ret
permno,yyyymm,


In [27]:
scalers = fit_scaler(df[df['s']==0],cols)
if len(scalers[2]):
    print("Some columns coudn't be scaled - ",scalers[2])
df_z = transform_group(df,scalers[0],scalers[1])
new_cols = {col: 'z' + col for col in scalers[1]}
df_z = df_z.rename(columns=new_cols)
df_z = df_z[sorted(df_z.columns, key=lambda col: 'z' in col)]
df_z = df_z.reset_index(drop=True)

In [29]:
C = df_z.groupby(['permno','yyyymm'])['zret'].count()
C[C>1]

,,zret
permno,yyyymm,


In [34]:
# Missing data Statistics
(df_z.isna().sum()/df_z.shape[0]).describe(percentiles=[0.01,0.1,0.5,0.9,0.99])

,0
count,49.000000
mean,0.243354
std,0.247184
min,0.000000
1%,0.000000
10%,0.000000
50%,0.188019
90%,0.624492
99%,0.820316
max,0.964158


In [35]:
# === impute with mean === #
df_mean = df_z.fillna(0)

# === impute with median === #
medians = df_z[df_z['s']==0].groupby(['year', 'ind']).median()

df_median = df_z.set_index(['year', 'ind'])
df_median = df_median.fillna(medians)
df_median = df_median.fillna(0)
df_median = df_median.reset_index()

In [36]:
col_missing = df_z.isna().sum().sort_values(ascending=False)
col_missing = col_missing[col_missing>0]

In [37]:
for i in range(1,len(col_missing)+1):
    R = 100*df_z.drop(columns = col_missing.index[:i]).dropna().shape[0]/df_z.shape[0]
    print(f"When we drop {i} columns we get {R:0.1f}%of the data")

When we drop 1 columns we get 1.8%of the data
When we drop 2 columns we get 2.0%of the data
When we drop 3 columns we get 5.9%of the data
When we drop 4 columns we get 7.5%of the data
When we drop 5 columns we get 7.7%of the data
When we drop 6 columns we get 9.5%of the data
When we drop 7 columns we get 9.6%of the data
When we drop 8 columns we get 14.4%of the data
When we drop 9 columns we get 22.6%of the data
When we drop 10 columns we get 29.1%of the data
When we drop 11 columns we get 33.3%of the data
When we drop 12 columns we get 34.2%of the data
When we drop 13 columns we get 41.1%of the data
When we drop 14 columns we get 42.4%of the data
When we drop 15 columns we get 43.5%of the data
When we drop 16 columns we get 45.9%of the data
When we drop 17 columns we get 46.3%of the data
When we drop 18 columns we get 55.3%of the data
When we drop 19 columns we get 68.0%of the data
When we drop 20 columns we get 72.9%of the data
When we drop 21 columns we get 72.9%of the data
When we 

In [38]:
# === Drop nans === #
mask1 = col_missing.index[:32]
mask2 = col_missing.index[:20]
dropped = df_z.drop(columns=mask1)
df_mean_mask = df_z.drop(columns=mask2).fillna(0)
dropped = dropped.dropna()

In [39]:
print(df_mean.shape)
print(df_mean_mask.shape)
print(dropped.shape)

(1629397, 49)
(1629397, 29)
(1449240, 17)


In [40]:
# === Save datasets === #
df_mean.to_parquet(data_dir + 'imputed/industry_mean.parquet')
df_mean_mask.to_parquet(data_dir + 'imputed/industry_mean_mask.parquet')
df_median.to_parquet(data_dir + 'imputed/industry_median.parquet')
dropped.to_parquet(data_dir + 'imputed/dropped.parquet')
df.to_parquet(data_dir + 'standardized_factors.parquet')